In [ ]:
import pandas as pd
from neuralforecast import NeuralForecast
from neuralforecast.models import NHITS, NBEATSx, DLinear, Autoformer, Informer, FEDformer
from neuralforecast.losses.pytorch import MAPE

def train_neuralforecast(df, target, date_column, test_start_date, horizon, lookback_window, hist_exog_list, futr_exog_list, index_column, epochs, granularity):
    df['unique_id'] = df[index_column].astype(str)
    df['y']         = df[target]
    df['ds']        = pd.to_datetime(df[date_column])
    df['ds']        = df['ds'].dt.to_period(granularity).dt.to_timestamp(granularity)

    train_df = df[df['ds'] < test_start_date]
    test_df  = df[df['ds'] >= test_start_date]

    # Trim columns
    features         = list(dict.fromkeys(['unique_id', 'ds', 'y'] + hist_exog_list + futr_exog_list))
    train_df         = train_df[features]
    test_df_original = test_df[['unique_id', 'ds', 'y'] + hist_exog_list]
    test_df_input    = test_df[['unique_id', 'ds']      + futr_exog_list]

    common_args = dict(
                        h=horizon,
                        input_size= lookback_window,
                        loss=MAPE(),
                        max_steps= epochs,
                        scaler_type='robust'
                        )
    
    models = [
                NHITS(**common_args,   hist_exog_list=hist_exog_list, futr_exog_list=futr_exog_list),
                NBEATSx(**common_args, hist_exog_list=hist_exog_list, futr_exog_list=futr_exog_list),
                DLinear(**common_args),
                Autoformer(**common_args, futr_exog_list=futr_exog_list),
                Informer(**common_args,   futr_exog_list=futr_exog_list),
                FEDformer(**common_args,  futr_exog_list=futr_exog_list)
            ]

    nf = NeuralForecast(models=models, freq=granularity)
    nf.fit(df=train_df)
    forecasts = nf.predict(futr_df=test_df_input)

    results = forecasts.merge(test_df_original, on=['unique_id', 'ds'], how='left')

    return results
